# Get normalisations

The `normalisation` is what we divide the change in emissions or concentration by to get the methane lifetime sensitivity to precursors. This will be the 1850-2014 changes for each precursor species to make the changes consistent with Thornhill et al. 2021a, 2021b which used AerChemMIP experiments that are differences between 1850 and 2014.

We have to specify a value for every atmospheric species, whether or not it is a methane-lifetime precursor species, since we sum over all species.

In [ ]:
import pandas as pd
import pooch

In [ ]:
rcmip_emissions_file = pooch.retrieve(
    url="doi:10.5281/zenodo.4589756/rcmip-emissions-annual-means-v5-1-0.csv",
    known_hash="md5:4044106f55ca65b094670e7577eaf9b3",
)

rcmip_concentration_file = pooch.retrieve(
    url=(
        "doi:10.5281/zenodo.4589756/"
        "rcmip-concentrations-annual-means-v5-1-0.csv"
    ),
    known_hash="md5:0d82c3c3cdd4dd632b2bb9449a5c315f",
)

In [ ]:
df_emis = pd.read_csv(rcmip_emissions_file)
df_conc = pd.read_csv(rcmip_concentration_file)

In [ ]:
slcfs = ['Sulfur', 'CO', 'VOC', 'NOx', 'BC', 'OC', 'NH3']
ghgs = ['CO2', 'CH4', 'N2O', 'CFC-11', 'CFC-12', 'CFC-113', 'CFC-114', 'CFC-115',
    'HCFC-22', 'HCFC-141b', 'HCFC-142b',
    'CCl4', 'CHCl3', 'CH2Cl2', 'CH3Cl', 'CH3CCl3', 'CH3Br',
    'Halon-1211', 'Halon-1301', 'Halon-2402',
    'CF4', 'C2F6', 'C3F8', 'c-C4F8', 'C4F10', 'C5F12', 'C6F14', 'C7F16', 'C8F18',
    'NF3', 'SF6', 'SO2F2',
    'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea', 'HFC-23', 'HFC-236fa', 'HFC-245fa', 'HFC-32',
    'HFC-365mfc', 'HFC-4310mee']

In [ ]:
normalisation = {}
for ispec, species in enumerate(slcfs):
    species_rcmip_name = species.replace("-", "")
    emis_in = df_emis.loc[
        (df_emis['Scenario']=='ssp245') & (df_emis['Variable'].str.endswith("|"+species_rcmip_name)) & 
        (df_emis['Region']=='World'), '1750':'2100'
    ].interpolate(axis=1).values.squeeze()
    normalisation[species] = emis_in[264] - emis_in[100]

for ispec, species in enumerate(ghgs):
    species_rcmip_name = species.replace("-", "")
    conc_in = df_conc.loc[
        (df_conc['Scenario']=='ssp245') & (df_conc['Variable'].str.endswith("|"+species_rcmip_name)) & 
        (df_conc['Region']=='World'), '1750':'2100'
    ].interpolate(axis=1).values.squeeze()
    normalisation[species] = conc_in[264] - conc_in[100]

In [ ]:
normalisation